In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense, Embedding
from tensorflow.keras.utils import to_categorical

In [2]:
# Sample words
words = ["cat", "car", "cap", "bat", "bar", "bag"]

# Create a set of unique characters
chars = sorted(list(set("".join(words))))
char_to_idx = {c: i for i, c in enumerate(chars)}
idx_to_char = {i: c for i, c in enumerate(chars)}

vocab_size = len(chars)
print("Characters:", chars)
print("Vocab size:", vocab_size)

Characters: ['a', 'b', 'c', 'g', 'p', 'r', 't']
Vocab size: 7


In [3]:
X, y = [], []
for word in words:
    for i in range(len(word) - 1):
        seq = word[:i+1]      # input sequence
        target = word[i+1]    # next character
        X.append([char_to_idx[c] for c in seq])
        y.append(char_to_idx[target])

In [4]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = max(len(seq) for seq in X)
X = pad_sequences(X, maxlen=max_len, padding='pre')
y = to_categorical(y, num_classes=vocab_size)

print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (12, 2)
y shape: (12, 7)


In [5]:
model_rnn = Sequential([
    Embedding(input_dim=vocab_size, output_dim=8, input_length=max_len),
    SimpleRNN(32, return_sequences=False),
    Dense(vocab_size, activation='softmax')
])

model_rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_rnn.summary()

C:\Users\bbuser\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [6]:
model_rnn.fit(X, y, epochs=300, verbose=0)

In [7]:
model_lstm = Sequential([
    Embedding(input_dim=vocab_size, output_dim=8, input_length=max_len),
    LSTM(32),
    Dense(vocab_size, activation='softmax')
])

model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.fit(X, y, epochs=300, verbose=0)

In [8]:
def predict_next_char(model, text):
    seq = [char_to_idx[c] for c in text]
    seq = pad_sequences([seq], maxlen=max_len, padding='pre')
    pred = model.predict(seq, verbose=0)
    next_char = idx_to_char[np.argmax(pred)]
    return next_char

for prefix in ["c", "ca", "b", "ba"]:
    print(f"{prefix} → {predict_next_char(model_lstm, prefix)}")

c → a
ca → r
b → a
ba → t
